# nltkの文書群にgensimのLDAを適用してみる

## 準備

必要ライブラリ(全部pipで入ります)
 * nltk
 * gensim
 * pyLDAvis

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis
!pip install --upgrade pandas==1.2

Requirement already up-to-date: pandas==1.2 in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [2]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## データロード・前処理

In [3]:
#dataset読み込み
from nltk.corpus import reuters as corpus

### 今回はこういう文書(をBOW化したもの)を用います

In [4]:
#必要に応じて以下のコードを実行してください．
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora


Archive:  /root/nltk_data/corpora/reuters.zip
replace /root/nltk_data/corpora/reuters/cats.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /root/nltk_data/corpora/reuters/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [6]:
#全document数
len(corpus.fileids())

10788

In [7]:

#前からk個のdocumentのみで学習する場合
k=1000
docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]

#全documentで学習する場合
# docs=[corpus.words(fileid) for fileid in corpus.fileids()]

# print(docs[:5])
# print("num of docs:", len(docs))

## 前処理 

In [8]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

# 一度LDAしてみる等して，適宜ノイズになってそうな記号等を見つけて，ストップワードリストに新たに加える
#【発展】記号や数字は正規表現で消してみましょう
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [9]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [10]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## LDA準備

In [11]:
import gensim
from gensim import corpora

In [12]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [13]:
#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'15': 0, '17': 1, '1985': 2, '30': 3, '300': 4, '53': 5, '95': 6, '>.': 7, 'able': 8, 'accounting': 9, 'action': 10, 'advantage': 11, 'allege': 12, 'allow': 13, 'also': 14, 'american': 15, 'among': 16, 'analyst': 17, 'april': 18, 'asia': 19, 'asian': 20, 'ask': 21, 'association': 22, 'australia': 23, 'australian': 24, 'avow': 25, 'await': 26, 'aware': 27, 'barrier': 28, 'beef': 29, 'beyond': 30, 'big': 31, 'block': 32, 'boost': 33, 'broker': 34, 'budget': 35, 'business': 36, 'businessmen': 37, 'button': 38, 'call': 39, 'canberra': 40, 'capel': 41, 'capital': 42, 'centre': 43, 'chairman': 44, 'chief': 45, 'co': 46, 'coal': 47, 'commercial': 48, 'complete': 49, 'concern': 50, 'conflict': 51, 'continue': 52, 'correspondent': 53, 'cost': 54, 'could': 55, 'country': 56, 'curb': 57, 'cut': 58, 'damage': 59, 'day': 60, 'defuse': 61, 'democratic': 62, 'deputy': 63, 'despite': 64, 'deterioration': 65, 'diplomatic': 66, 'director': 67, 'disadvantage': 68, 'dispute': 69, 'domestically': 70, 'due

In [14]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus_[0][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1)]


In [15]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(16, 1), (19, 1), (20, 1), (37, 1), (55, 1), (59, 2), (72, 1), (85, 1), (88, 1), (89, 1), (96, 1), (116, 1), (120, 2), (142, 1), (157, 1), (198, 1), (209, 1), (210, 1), (256, 1)]


## LDA学習

In [16]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [17]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.009*"stock" + 0.008*"bank" + 0.006*"corp" + 0.006*"may" + 0.006*"would" + 0.005*"one" + 0.004*"15" + 0.004*"two" + 0.004*"record" + 0.004*"pay"')
(1, '0.009*"87" + 0.008*"corp" + 0.008*"april" + 0.006*"qtr" + 0.005*"sales" + 0.005*"15" + 0.004*"13" + 0.004*"rev" + 0.004*"inc" + 0.004*"1st"')
(2, '0.025*"87" + 0.014*"09" + 0.012*"export" + 0.007*"04" + 0.007*"1985" + 0.006*"trade" + 0.006*"03" + 0.006*"import" + 0.005*"japan" + 0.004*"month"')
(3, '0.010*"trade" + 0.009*"japan" + 0.007*"would" + 0.005*"market" + 0.005*"industry" + 0.005*"export" + 0.004*"japanese" + 0.004*"state" + 0.004*"dollar" + 0.004*"deficit"')
(4, '0.006*"market" + 0.005*"sales" + 0.005*"trade" + 0.005*"would" + 0.004*"buy" + 0.004*"source" + 0.004*"inc" + 0.004*"export" + 0.004*"corp" + 0.004*"last"')
(5, '0.005*"unit" + 0.004*"agree" + 0.003*"dollar" + 0.003*"stockpile" + 0.003*"buy" + 0.003*"would" + 0.003*"export" + 0.003*"right" + 0.003*"one" + 0.003*"market"')
(6, '0.005*"quarter" + 0.004*"corp" + 0.0

In [18]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(3, 0.6696013), (4, 0.042571172), (13, 0.07702938), (17, 0.20692326)]
document ID 1:[(10, 0.9666606)]
document ID 2:[(11, 0.98303235)]
document ID 3:[(2, 0.6655929), (15, 0.31525388)]
document ID 4:[(3, 0.13150138), (6, 0.8504953)]
document ID 5:[(4, 0.5240437), (16, 0.460025)]
document ID 6:[(3, 0.99198157)]
document ID 7:[(6, 0.9406139)]
document ID 8:[(1, 0.972853)]
document ID 9:[(11, 0.8596419), (17, 0.13549237)]


In [19]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [20]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(3, 0.66983396), (4, 0.04219089), (13, 0.07755467), (17, 0.20654556)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen 

## 可視化

In [21]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [25]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [23]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [24]:
!ls

sample_data  vis.html
